<a href="https://colab.research.google.com/github/juli-ter/Course3_work/blob/main/%D0%9F%D1%80%D0%B5%D0%B4%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 8.2 MB 17.7 MB/s 


In [3]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
import pymorphy2
import re
import string
from tqdm import tqdm
from collections import Counter
import numpy as np

from gensim import corpora
from datetime import datetime

tqdm.pandas()
nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

from multiprocessing import Pool
morph = pymorphy2.MorphAnalyzer()

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from IPython.display import clear_output
!pip install twarc #Twarc
!pip install tweepy # Tweepy 3.8.0
!pip install argparse #Argparse 3.2
!pip install xtract #Xtract 0.1 a3
!pip install wget #Wget 3.2
clear_output()

In [6]:
import gzip
import shutil
import os
import wget
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
path = "/content/drive/MyDrive/Course_3/"


### Базовая очистка

In [9]:
tweets_ru = pd.read_csv(path+'Text/hydratued_tweets_ru.csv')
tweets_ru = tweets_ru.drop(columns=['favorite_count', 'source', 'in_reply_to_screen_name', 'is_retweet', 'retweet_count'])
tweets_ru

,text,created_at,id_str
0,@lentaruofficial Что значит после окончания са...,Sun May 03 10:49:33 +0000 2020,1256898686115864582
1,А помните в Ухане мужчина с короновирусом в ра...,Wed Apr 22 12:50:58 +0000 2020,1252942974377496576
2,"выходила выкинуть мусор под домом, а дед-сосед...",Wed Apr 15 18:30:08 +0000 2020,1250491613383725057
3,"Меня бесит слово ""коронавирус"" уже с самого ег...",Tue May 05 09:08:05 +0000 2020,1257597927964446720
4,топ коммент обсуждения что раскупают в магазин...,Tue Mar 17 09:06:58 +0000 2020,1239840640286044161
...,...,...,...
160946,@promadm @SamMobilePriem @Gennadi070786 @D_Aza...,Wed Apr 15 04:44:36 +0000 2020,1250283862522916866
160947,"Так в России, это нормально, когда долбоебы пр...",Wed Apr 15 10:25:34 +0000 2020,1250369668122054656
160948,Коротко о ситуации с короновирусом в моем горо...,Sun Mar 15 17:09:07 +0000 2020,1239237201290018816
160949,"Шел какой там день карантина, я почти купила с...",Wed Apr 22 19:15:04 +0000 2020,1253039638316486658


In [10]:
tweets_en = pd.read_csv(path+'Text/hydratued_tweets_en.csv')
tweets_en = tweets_en.drop(columns=['favorite_count', 'source', 'in_reply_to_screen_name', 'is_retweet', 'retweet_count'])
tweets_en

,text,created_at,id_str
0,umair haque asks if life will go back to norma...,Mon May 11 17:13:00 +0000 2020,1259894286285496322
1,“Do as I say not as I do” miniTrump https://t....,Thu Apr 30 01:40:43 +0000 2020,1255673406143500288
2,Cities under lockdown amid Covid-19 scare: Sce...,Mon Mar 23 08:49:20 +0000 2020,1242010532112216064
3,#SackBorisAndDom #COVID19 #Conservative #KierS...,Mon May 25 18:05:05 +0000 2020,1264980823956246531
4,"No, Governor Kemp, I won’t open my Georgia res...",Fri Apr 24 17:21:24 +0000 2020,1253735808961478657
...,...,...,...
152816,IF only that was trumps intention https://t.co...,Sat Apr 25 08:49:28 +0000 2020,1253969364228898816
152817,Lizzo out there having the time of her life in...,Wed Mar 18 22:30:46 +0000 2020,1240405312089227265
152818,Day Job: Cory Booker’s Top Aide. His Night One...,Thu May 07 15:27:58 +0000 2020,1258418305003487233
152819,Remember: F*CK @speakvos @SenFitzgerald @mkego...,Sun Apr 12 23:42:11 +0000 2020,1249482980629917701


In [10]:
!pip install emoji

     |████████████████████████████████| 185 kB 13.9 MB/s 
  Created wheel for emoji: filename=emoji-1.5.0-py3-none-any.whl size=187457 sha256=64af7ed55975acde8e04e922c631cd5c44a04a29be2d41f2ee5920f96d39cf5a
  Stored in directory: /root/.cache/pip/wheels/db/b5/f6/b39abf14e94b3d6640613bbe630a66c10ccf7a12882d064fb5
Successfully built emoji


In [11]:
import emoji

In [11]:
def deEmojify(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

punctuation = string.punctuation + '…«»—–'
def clean_text(text):
# Make text lowercase
    text = text.lower()
# remove text after @
    text = re.sub(r'@\S+', '', text)
# remove urls
    text = re.sub(r"http\S+", ' ', text)
# remove emojies
    # text = give_emoji_free_text(text)
# remove punctuation   
    text = re.sub(r'[%s]' % re.escape(punctuation), ' ', text)
# remove numbers
    text = re.sub(r'[0-9]', ' ', text)
# remove non letters
    text = re.sub(r'[^\w\s]', '', text)
# remove dublicate spaces
    text = re.sub('\s\s+', ' ', text)
#remove new line
    text = re.sub('(\r\n|\r|\n)', ' ', text)
#remove nonbreakable space
    text = re.sub(u'\xa0', ' ', text)
    return text


df_clean_ru = pd.DataFrame(tweets_ru.text.progress_apply(lambda x: clean_text(x)))
df_clean_ru

100%|██████████| 160951/160951 [00:03<00:00, 40489.10it/s]


,text
0,что значит после окончания самоизоляции иш ра...
1,а помните в ухане мужчина с короновирусом в ра...
2,выходила выкинуть мусор под домом а дед сосед ...
3,меня бесит слово коронавирус уже с самого его ...
4,топ коммент обсуждения что раскупают в магазин...
...,...
160946,шверника а обработка территории и подъезда не...
160947,так в россии это нормально когда долбоебы прин...
160948,коротко о ситуации с короновирусом в моем горо...
160949,шел какой там день карантина я почти купила се...


In [12]:
df_clean_en = pd.DataFrame(tweets_en.text.progress_apply(lambda x: clean_text(x)))
df_clean_en

100%|██████████| 152821/152821 [00:03<00:00, 41210.22it/s]


,text
0,umair haque asks if life will go back to norma...
1,do as i say not as i do minitrump
2,cities under lockdown amid covid scare scenes ...
3,sackborisanddom covid conservative kierstarme...
4,no governor kemp i wont open my georgia restau...
...,...
152816,if only that was trumps intention
152817,lizzo out there having the time of her life in...
152818,day job cory bookers top aide his night one e ...
152819,remember f ck they do not care about the voter...


### Стоп слова и лемматизация

In [14]:

stopwords_list = stopwords.words('russian') + stopwords.words('english')
stopwords_list.extend(('её', 'это', 'всё', 'ещё', 'весь', 'свой', 'который', 'мочь', ''))
print(stopwords_list)

def lemmatize(text, lemmer = morph, stopwords = stopwords_list):
    words = text.split(' ')
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if len(w) >= 3 and not w in stopwords]

with Pool(processes = 4) as pool:
    lemmatized_ru = pool.starmap(lemmatize, zip(df_clean_ru.text))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [15]:
comments_lemmatized_ru = pd.DataFrame({'text' : lemmatized_ru, 'date' : tweets_ru.created_at, 'raw_text':tweets_ru.text})
comments_lemmatized_ru

,text,date,raw_text
0,"[значит, окончание, самоизоляция, размечтаться]",Sun May 03 10:49:33 +0000 2020,@lentaruofficial Что значит после окончания са...
1,"[помнить, ухань, мужчина, короновирус, раскали...",Wed Apr 22 12:50:58 +0000 2020,А помните в Ухане мужчина с короновирусом в ра...
2,"[выходить, выкинуть, мусор, дом, дед, сосед, н...",Wed Apr 15 18:30:08 +0000 2020,"выходила выкинуть мусор под домом, а дед-сосед..."
3,"[бесить, слово, коронавирус, появление, жизнь]",Tue May 05 09:08:05 +0000 2020,"Меня бесит слово ""коронавирус"" уже с самого ег..."
4,"[топ, коммент, обсуждение, раскупать, магазин,...",Tue Mar 17 09:06:58 +0000 2020,топ коммент обсуждения что раскупают в магазин...
...,...,...,...
160946,"[шверник, обработка, территория, подъезд, пров...",Wed Apr 15 04:44:36 +0000 2020,@promadm @SamMobilePriem @Gennadi070786 @D_Aza...
160947,"[россия, нормально, долбоеб, принимать, решение]",Wed Apr 15 10:25:34 +0000 2020,"Так в России, это нормально, когда долбоебы пр..."
160948,"[коротко, ситуация, короновирус, город, массов...",Sun Mar 15 17:09:07 +0000 2020,Коротко о ситуации с короновирусом в моем горо...
160949,"[идти, день, карантин, купить, краска, волос, ...",Wed Apr 22 19:15:04 +0000 2020,"Шел какой там день карантина, я почти купила с..."


In [16]:

with Pool(processes = 4) as pool:
    lemmatized_en = pool.starmap(lemmatize, zip(df_clean_en.text))

In [17]:
comments_lemmatized_en = pd.DataFrame({'text' : lemmatized_en, 'date' : tweets_en.created_at, 'raw_text':tweets_en.text})
comments_lemmatized_en

,text,date,raw_text
0,"[umair, haque, asks, life, back, normal, coron...",Mon May 11 17:13:00 +0000 2020,umair haque asks if life will go back to norma...
1,"[say, minitrump]",Thu Apr 30 01:40:43 +0000 2020,“Do as I say not as I do” miniTrump https://t....
2,"[cities, lockdown, amid, covid, scare, scenes,...",Mon Mar 23 08:49:20 +0000 2020,Cities under lockdown amid Covid-19 scare: Sce...
3,"[sackborisanddom, covid, conservative, kiersta...",Mon May 25 18:05:05 +0000 2020,#SackBorisAndDom #COVID19 #Conservative #KierS...
4,"[governor, kemp, wont, open, georgia, restaura...",Fri Apr 24 17:21:24 +0000 2020,"No, Governor Kemp, I won’t open my Georgia res..."
...,...,...,...
152816,"[trumps, intention]",Sat Apr 25 08:49:28 +0000 2020,IF only that was trumps intention https://t.co...
152817,"[lizzo, time, life, florida]",Wed Mar 18 22:30:46 +0000 2020,Lizzo out there having the time of her life in...
152818,"[day, job, cory, bookers, top, aide, night, on...",Thu May 07 15:27:58 +0000 2020,Day Job: Cory Booker’s Top Aide. His Night One...
152819,"[remember, care, voters, care]",Sun Apr 12 23:42:11 +0000 2020,Remember: F*CK @speakvos @SenFitzgerald @mkego...


### Уберем нечастые, маленькие

In [18]:
def wordsCount(texts):
    count = Counter()
    for words in texts:
        for word in words:
            count[word] += 1
    return count

def removeNotCommonWords(text, word_counter):
    return [word for word in text if word_counter[word] > 10]

def func(df):
    word_counter = wordsCount(df.text.values.tolist())
    df.text = df.text.apply(lambda x: removeNotCommonWords(x, word_counter))
    df.text = df.text.apply(lambda x: ' '.join(x))
    df.text = df.text.apply(lambda x: x.split())
    df = df[df.text.apply(lambda x: len(x) >= 4)]
    return df



In [19]:
clean_texts_ru = func(comments_lemmatized_ru)
clean_texts_ru.to_csv(path+'clean_texts_ru.csv', index = False)
clean_texts_ru

,text,date,raw_text
1,"[помнить, ухань, мужчина, короновирус, масло, ...",Wed Apr 22 12:50:58 +0000 2020,А помните в Ухане мужчина с короновирусом в ра...
2,"[выходить, выкинуть, мусор, дом, дед, сосед, н...",Wed Apr 15 18:30:08 +0000 2020,"выходила выкинуть мусор под домом, а дед-сосед..."
3,"[бесить, слово, коронавирус, появление, жизнь]",Tue May 05 09:08:05 +0000 2020,"Меня бесит слово ""коронавирус"" уже с самого ег..."
4,"[топ, коммент, обсуждение, раскупать, магазин,...",Tue Mar 17 09:06:58 +0000 2020,топ коммент обсуждения что раскупают в магазин...
5,"[действительно, делаться, очень, многое, прода...",Fri Mar 27 18:18:40 +0000 2020,"@dima_29ss @sd100674 Действительно, делается о..."
...,...,...,...
160945,"[никто, сломать, таджик, носитель, корона]",Sun Mar 08 08:00:31 +0000 2020,"@jksbabyboo Нас никогда никто не сломает, мы-т..."
160946,"[обработка, территория, подъезд, проводить, ка...",Wed Apr 15 04:44:36 +0000 2020,@promadm @SamMobilePriem @Gennadi070786 @D_Aza...
160947,"[россия, нормально, долбоеб, принимать, решение]",Wed Apr 15 10:25:34 +0000 2020,"Так в России, это нормально, когда долбоебы пр..."
160948,"[коротко, ситуация, короновирус, город, массов...",Sun Mar 15 17:09:07 +0000 2020,Коротко о ситуации с короновирусом в моем горо...


In [20]:
clean_texts_en = func(comments_lemmatized_en)
clean_texts_en.to_csv(path+'clean_texts_en.csv', index = False)
clean_texts_en

,text,date,raw_text
0,"[asks, life, back, normal, coronavirus]",Mon May 11 17:13:00 +0000 2020,umair haque asks if life will go back to norma...
2,"[cities, lockdown, amid, covid, scare, scenes,...",Mon Mar 23 08:49:20 +0000 2020,Cities under lockdown amid Covid-19 scare: Sce...
3,"[covid, conservative, nhs, need, self, test]",Mon May 25 18:05:05 +0000 2020,#SackBorisAndDom #COVID19 #Conservative #KierS...
4,"[governor, kemp, wont, open, georgia, restaura...",Fri Apr 24 17:21:24 +0000 2020,"No, Governor Kemp, I won’t open my Georgia res..."
5,"[wisconsin, coronavirus, cases, rise, nearly, ...",Sun May 31 00:17:34 +0000 2020,Wisconsin coronavirus cases rise by nearly 20 ...
...,...,...,...
152814,"[health, covid, enforcement, statistics, infor...",Fri Apr 17 14:50:26 +0000 2020,(Health) COVID-19 enforcement statistics infor...
152815,"[coronavirus, confirmed, cases, double, china,...",Tue Mar 31 16:37:19 +0000 2020,"""Coronavirus: US confirmed cases now double th..."
152818,"[day, job, top, aide, night, one, volunteer, l...",Thu May 07 15:27:58 +0000 2020,Day Job: Cory Booker’s Top Aide. His Night One...
152819,"[remember, care, voters, care]",Sun Apr 12 23:42:11 +0000 2020,Remember: F*CK @speakvos @SenFitzgerald @mkego...
